This notebook shows the behavior of the implemented models using a largely reduced dataset (taking only 10 time step for demand data).

In [20]:
# Load modules
include("../src/simulation.jl")
include("../src/utils.jl")
include("../src/optimization.jl")
using .Optimization
using .Utils
using .Simulation

In [2]:
# Load data
data_dir = "/home/esteban_nb/dirhw/mae573-term-project/data"
data = Utils.load_data(data_dir, shrink=true)

Loading data from /home/esteban_nb/dirhw/mae573-term-project/data...

----- Dataset Information -----

Dataset: Bus
  Rows: 120
  Columns: 5
  Column Names: ["bus_id", "zone_id", "pd", "qd", "idx"]

Dataset: Branch
  Rows: 738
  Columns: 27
  Column Names: ["start_bus", "end_bus", "branch_id", "r", "x", "b", "ratea", "rateb", "ratec", "ratio", "angle", "status", "angmin", "angmax", "pf", "qf", "pt", "qt", "mu_sf", "mu_st", "mu_angmin", "mu_angmax", "branch_device_type", "interconnect", "sus", "start_idx", "end_idx"]

Dataset: Plant
  Rows: 548
  Columns: 33
  Column Names: ["plant_id", "bus_id", "pg", "qg", "qmax", "qmin", "vg", "mbase", "status", "pmax", "pmin", "pc1", "pc2", "qc1min", "qc1max", "qc2min", "qc2max", "ramp_agc", "ramp_10", "ramp_30", "ramp_q", "apf", "mu_pmax", "mu_pmin", "mu_qmax", "mu_qmin", "type", "interconnect", "genfuelcost", "geniob", "genioc", "geniod", "bus_idx"]

Dataset: Load
  Rows: 10
  Columns: 118
  Column Names: ["Date", "Time", "1079", "2002", "2011", "

Dict{String, DataFrames.DataFrame} with 5 entries:
  "branch"  => 738×27 DataFrame…
  "load"    => 10×118 DataFrame…
  "bus"     => 120×5 DataFrame…
  "plant"   => 548×33 DataFrame…
  "gencost" => 548×9 DataFrame…

In [3]:
# Simple DCOPF
using JuMP

println("Executing simple DCOPF...\n")
results = Optimization.perform_dcopf(data)

Executing simple DCOPF...

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2575217
Academic license 2575217 - for non-commercial use only - registered to en___@princeton.edu
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2575217 - for non-commercial use only - registered to en___@princeton.edu
Optimize a model with 23934 rows, 19540 columns and 73078 nonzeros
Model fingerprint: 0x24f289e5
Model has 4130 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [7e+00, 4e+01]
  QObjective range [2e-03, 4e-03]
  Bounds range     [4e-01, 1e+03]
  RHS range        [3e-01, 4e+03]
Presolve removed 19187 rows and 14359 columns
Presolve time: 0.05s
Presolved: 4747 rows, 8898 columns, 24131 nonzeros
Preso

(generation = 548×2 DataFrame
 Row │ plant  total_generation 
     │ Int64  Float64          
─────┼─────────────────────────
   1 │     1           2110.0
   2 │     2            343.0
   3 │     3            825.0
   4 │     4            100.0
   5 │     5           1995.0
   6 │     6           1800.0
   7 │     7           1242.0
   8 │     8           2780.0
   9 │     9            900.0
  10 │    10           1475.0
  11 │    11           1219.0
  ⋮  │   ⋮           ⋮
 539 │   539           3100.46
 540 │   540           3100.46
 541 │   541           3100.46
 542 │   542             12.0
 543 │   543            717.9
 544 │   544             66.0
 545 │   545            523.8
 546 │   546            523.8
 547 │   547            315.0
 548 │   548            294.6
               527 rows omitted, angles = 120×2 DataFrame
 Row │ bus    average_angle 
     │ Int64  Float64       
─────┼──────────────────────
   1 │     1        0.0
   2 │     2       33.5551
   3 │     3       -7.

In [4]:
num_scenarios = 10
scenarios = Simulation.generate_scenarios(data, num_scenarios, 0.1, 0.1)
probabilities = fill(1/num_scenarios, num_scenarios)

10-element Vector{Float64}:
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1
 0.1

In [21]:
include("../src/optimization.jl")
using .Optimization

# should have a unique set of decision that is minimimzing the objective
# whereas with heuristics you do take different actions in different scenarios and you take the most taken
model = Optimization.perform_stochastic_dcopf(data, scenarios, probabilities)

Hey!


LoadError: KeyError: key "slack_bus" not found